In [2]:
"""將tensorflow檔轉成tensorflow lite，使機器負載量變小"""
from tensorflow import lite
from tensorflow.keras import models

# Parameters
keras_model_filename = './h5/recording16.h5' #訓練好的模型
tflite_filename = './tflite/recording16.tflite' #預建置檔案

# Convert model to TF Lite model
model = models.load_model(keras_model_filename) #載入本來的模型
converter = lite.TFLiteConverter.from_keras_model(model) #將模型載入轉換器
tflite_model = converter.convert() #進行轉換
open(tflite_filename, 'wb').write(tflite_model) #輸出轉換後的模型


INFO:tensorflow:Assets written to: C:\Users\r6017\AppData\Local\Temp\tmp3nzah2ih\assets


INFO:tensorflow:Assets written to: C:\Users\r6017\AppData\Local\Temp\tmp3nzah2ih\assets


187824

In [15]:
#num_mfcc = 23，rec_duration = 0.5
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa
#import RPi.GPIO as GPIO

# Parameters
debug_time = 0 #Debug用
debug_acc = 1 #Debug用
led_pin = 8 #LED PIN
word_threshold = 0.5 #預測值>0.5，表示stop
rec_duration = 0.5 #每一段錄音持續時間
#window_stride = 0.5
sample_rate = 48000 #取樣率(依MIC不同而改變)
resample_rate = 8000 #重整後的取樣率(符合MODEL)
num_channels = 1 #音訊深度
num_mfcc = 23 #回傳mfcc的量
model_path = './tflite/recording23.tflite'
words = ['ㄏㄧㄡ', 'ㄟ', '吼', '啦', '嗯', '的一個', '的這個', '的那個', '著', '那', '那那個', '阿']#答案對應到的字詞

s = 0 #秒
m = 0 #分
h = 0 #時

# Sliding window
window = np.zeros(int(rec_duration * resample_rate) * 2)#取樣音頻數據變數

# GPIO 
#GPIO.setwarnings(False)
#GPIO.setmode(GPIO.BOARD)
#GPIO.setup(8, GPIO.OUT, initial=GPIO.LOW)

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#開始計時
counter = QTime()
counter.restart()

# Decimate (filter and downsample)
def decimate(signal, old_fs, new_fs):
    
    #檢查是否降低音頻
    if new_fs > old_fs:
        print("Error: target sample rate higher than original")
        return signal, old_fs
    
    #檢查是否為整數(只能在整數下執行)
    dec_factor = old_fs / new_fs
    if not dec_factor.is_integer():
        print("Error: can only decimate by integer factor")
        return signal, old_fs

    # Do decimation
    resampled_signal = scipy.signal.decimate(signal, int(dec_factor))
    
    return resampled_signal, new_fs

# This gets called every 0.5 seconds
def sd_callback(rec, frames, time, status):

    #GPIO.output(led_pin, GPIO.LOW)
    
    # Notify if errors
    if status:
        print('Error:', status)
    
    # Remove 2nd dimension from recording sample
    #壓縮成1D張量
    rec = np.squeeze(rec)
    
    # Resample
    #重取樣成8000HZ(以符合訓練模型)
    rec, new_fs = decimate(rec, sample_rate, resample_rate)
    
    # Save recording onto sliding window
    #將音訊輸入到window
    window[:len(window)//2] = window[len(window)//2:]
    window[len(window)//2:] = rec
    
    S = np.abs(librosa.stft(window))
    #print("np.sum(S):" + str(np.sum(S)))

    if np.sum(S) > 3000:
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=new_fs, #取樣率
                                            winlen=0.256, #音框涵蓋時間
                                            winstep=0.050, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=2048,#FFT大小
                                            preemph=0.0,#不用預強化濾波器
                                            ceplifter=0,#ROBUST
                                            appendEnergy=False,#係數0的話對被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if(list_val_max >= 0.3):#如果預測值>=0.3
            print(words[list_val_maxIndex])#輸出相對應的字詞
            print("MAX:" + str(list_val_max))#輸出預測值當中最大的值
            print(str(h) + "時" + str(m) + "分" + "{:.1f}秒".format(s))              
    
    
# Start streaming from microphone
with sd.InputStream(channels=num_channels,
                    samplerate=sample_rate,
                    blocksize=int(sample_rate * rec_duration),
                    callback=sd_callback):
    while True:
        s = float(counter.elapsed() / 1000)
        if(s >= 60):
            counter.restart()
            s = 0
            m = m + 1
        if(m == 60):
            m = 0
            h = h + 1
        pass


著
MAX:0.8246859312057495
0時0分2.6秒
著
MAX:0.921787679195404
0時0分3.5秒
那
MAX:0.4175722599029541
0時0分5.5秒
著
MAX:0.5986005067825317
0時0分6.1秒
啦
MAX:0.7231849431991577
0時0分7.1秒
著
MAX:0.5769041776657104
0時0分9.5秒
嗯
MAX:0.618951678276062
0時0分10.0秒
那那個
MAX:0.510421872138977
0時0分10.5秒
著
MAX:0.7175104022026062
0時0分14.6秒
嗯
MAX:0.7754087448120117
0時0分15.2秒
著
MAX:0.8363051414489746
0時0分16.1秒
著
MAX:0.8798959851264954
0時0分18.2秒
嗯
MAX:0.6573458909988403
0時0分19.0秒
那
MAX:0.7003562450408936
0時0分21.7秒
著
MAX:0.9144920110702515
0時0分22.1秒
嗯
MAX:0.7604119777679443
0時0分22.7秒
著
MAX:0.6179332137107849
0時0分24.6秒
著
MAX:0.3545456528663635
0時0分25.2秒
嗯
MAX:0.7118475437164307
0時0分26.2秒
著
MAX:0.5517088174819946
0時0分26.7秒
著
MAX:0.8381718993186951
0時0分29.2秒
著
MAX:0.6952312588691711
0時0分29.8秒
著
MAX:0.8381347060203552
0時0分32.2秒
著
MAX:0.884494423866272
0時0分32.8秒
著
MAX:0.7520496845245361
0時0分33.2秒
啦
MAX:0.7338104844093323
0時0分34.7秒
的那個
MAX:0.4631894826889038
0時0分35.3秒
那
MAX:0.5509160161018372
0時0分37.2秒
那那個
MAX:0.519541323184967


KeyboardInterrupt: 

In [14]:
#num_mfcc = 23，rec_duration = 0.25
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa
#import RPi.GPIO as GPIO

# Parameters
debug_time = 0 #Debug用
debug_acc = 1 #Debug用
led_pin = 8 #LED PIN
word_threshold = 0.5 #預測值>0.5，表示stop
rec_duration = 1 #每一段錄音持續時間
#window_stride = 0.5
sample_rate = 8000 #取樣率(依MIC不同而改變)
resample_rate = 8000 #重整後的取樣率(符合MODEL)
num_channels = 1 #音訊深度
num_mfcc = 23 #回傳mfcc的量
model_path = './tflite/recording23.tflite'
words = ['ㄏㄧㄡ', 'ㄟ', '吼', '啦', '嗯', '的一個', '的這個', '的那個', '著', '那', '那那個', '阿']#答案對應到的字詞

s = 0 #秒
m = 0 #分
h = 0 #時

# Sliding window
window = np.zeros(int(rec_duration * resample_rate) * 1)#取樣音頻數據變數

# GPIO 
#GPIO.setwarnings(False)
#GPIO.setmode(GPIO.BOARD)
#GPIO.setup(8, GPIO.OUT, initial=GPIO.LOW)

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#開始計時
counter = QTime()
counter.restart()

# Decimate (filter and downsample)
def decimate(signal, old_fs, new_fs):
    
    #檢查是否降低音頻
    if new_fs > old_fs:
        print("Error: target sample rate higher than original")
        return signal, old_fs
    
    #檢查是否為整數(只能在整數下執行)
    dec_factor = old_fs / new_fs
    if not dec_factor.is_integer():
        print("Error: can only decimate by integer factor")
        return signal, old_fs

    # Do decimation
    resampled_signal = scipy.signal.decimate(signal, int(dec_factor))
    
    return resampled_signal, new_fs

# This gets called every 0.5 seconds
def sd_callback(rec, frames, time, status):

    #GPIO.output(led_pin, GPIO.LOW)
    
    # Notify if errors
    if status:
        print('Error:', status)
    
    # Remove 2nd dimension from recording sample
    #壓縮成1D張量
    rec = np.squeeze(rec)
    
    # Resample
    #重取樣成8000HZ(以符合訓練模型)
    rec, new_fs = decimate(rec, sample_rate, resample_rate)
    
    # Save recording onto sliding window
    #將音訊輸入到window
    #window[:2000] = window[2000:4000]
    #window[2000:4000] = window[4000:6000]
    #window[4000:6000] = window[6000:]
    #window[6000:] = rec
    window = rec
    
    S = np.abs(librosa.stft(window))
    #print("np.sum(S):" + str(np.sum(S)))

    if np.sum(S) > 3500:
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=new_fs, #取樣率
                                            winlen=0.256, #音框涵蓋時間
                                            winstep=0.050, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=2048,#FFT大小
                                            preemph=0.0,#不用預強化濾波器
                                            ceplifter=0,#ROBUST
                                            appendEnergy=False,#係數0的話對被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if(list_val_max >= 0.3):#如果預測值>=0.3
            print(words[list_val_maxIndex])#輸出相對應的字詞
            print("MAX:" + str(list_val_max))#輸出預測值當中最大的值
            print(str(h) + "時" + str(m) + "分" + "{:.1f}秒".format(s))
    
# Start streaming from microphone
with sd.InputStream(channels=num_channels,
                    samplerate=sample_rate,
                    blocksize=int(sample_rate * rec_duration),
                    callback=sd_callback):
    while True:
        s = float(counter.elapsed() / 1000)
        if(s >= 60):
            counter.restart()
            s = 0
            m = m + 1
        if(m == 60):
            m = 0
            h = h + 1
        pass


ㄏㄧㄡ
MAX:0.5631705522537231
0時0分3.5秒
著
MAX:0.5855104923248291
0時0分7.6秒
著
MAX:0.40299466252326965
0時0分11.7秒
ㄏㄧㄡ
MAX:0.9750849604606628
0時0分14.5秒
ㄏㄧㄡ
MAX:0.7813223600387573
0時0分17.5秒
ㄏㄧㄡ
MAX:0.30224084854125977
0時0分20.2秒
ㄏㄧㄡ
MAX:0.9357806444168091
0時0分23.6秒
ㄏㄧㄡ
MAX:0.8721511363983154
0時0分26.4秒
的這個
MAX:0.9593712687492371
0時0分29.5秒
著
MAX:0.6996145844459534
0時0分33.5秒


KeyboardInterrupt: 